In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Debugging a Learning Algorithm
After running linear regression we find an unacceptable accuracy. We can
* Get more training examples (can get out of hand): fixes high variance
* Try a smaller set of features to prevent overfitting: fixes high variance
* Try getting additional features (can get out of hand): fixes high bias
* Try adding polynomial features: fixes high bias
* Try decreasing $\lambda$: fixes high bias
* Try increasing $\lambda$: fixes high variance

## Diagnostic
* A test you can run to gain insight into what is/isn't working in a learning algorithm, and gain guidance as to how best to improve its performance.
* Can take time to implement, but can be valuable.

# Evaluating a Hypothesis
* Split the training set into **training** and **test sets**, perhaps 70/30%.
* If the data is in any way ordered, it's better to randomly split the training set.
* For regression we can calculate the sum of squares error for the test set
$$
J_{test}(\Theta) = \frac{1}{2m_{test}}\sum_{i=1}^{m_{test}} (h_{\Theta}^{(i)} - y^{(i)}_{test})^2
$$
* For classification we can use **0/1 Misclassification Error**: the proportion of misclassified test data

# Model Selection
Given many models with different polynomial degrees we can
* Split the training data into training, **validation**, and test sets (perhaps 60/20/20%)
* Optimize parameters $\Theta$ for each model
* Find the polynomial degree $d$ that yields the least error **on the validation set**
* Estimate the error using the test set for $J_{test}(\Theta), d = \text{theta from polynomial with lower error}$

This approach prevents fitting the polynomial degree $d$ to the test set.

# Diagnosing Bias vs Variance
Not generalizing well is a variance problem. Not fitting the data well is a bias problem. We want a model in between.

## Select Polynomial Degree $d$
We need to determine whether bias (underfitting) or variance (overfitting) is leading to a high rate of misclassified test data.  

The training error will tend to decrease as we increase polynomial degree $d$.
The validation error will tend to decrease as we increase $d$ to a point, and then it will increase as $d$ is increased, forming a convex curve.

There is an optimal value of $d$ that is at the **global minimum of the validation error accuracy curve**.

* TODO: plot $d$ vs $J_{training}(\Theta)$ and $J_{validation}(\Theta)$

## Select $\lambda$ For Bias/Variance

* Large $\lambda$ $\rightarrow$ high bias/underfit
* Small $\lambda$ $\rightarrow$ high variance/overfit
* Somewhere in-between $\rightarrow$ reasonable fit

The training error will tend to increase as we increase $\lambda$.  
The validation error will tend to decrease as we increase $d$ to a point, and then it will increase as $d$ is increased, forming a convex curve.

There is an optimal value of $\lambda$ that is at the **global minimum of the validation error accuracy curve**.

* TODO: plot $\lambda$ vs $J_{training}(\Theta)$ and $J_{validation}(\Theta)$

### Algorithm
1. Create a list of lambdas (i.e. $\lambda \in {0, 2^{0,1,\ldots,10}/100}$.
2. Create a set of models with different degrees or any other variants.
3. Iterate through the $\lambda$s and for each $\lambda$ go through all the models to learn some $\Theta$.
4. Compute the cross validation error using the learned $\Theta$ (computed with $\lambda$) on the $J_{validation}(\Theta)$ without regularization or $\lambda = 0$.
5. Select the best combo that produces the lowest error on the cross validation set.
6. Using the best combo $\Theta$ and $\lambda$, apply it on $J_{test}(\Theta)$ to see if it has a good generalization of the problem.

## Determine Whether More Data Will Help
For data with many features it is easy to visualize the data and model. We can generate **learning curves** that can help to debug a learning algorithm even if it is not easy to visualize the data.

We can plot the size of the training set $m$ vs $J_{training}(\Theta)$ and $J_{validation}(\Theta)$.

### High Bias
If algorithm is suffering from bias (underfitting), it's harder to fit all the training set examples perfectly, so $J_{training}(\Theta)$ increases as we get more data. $J_{validation}(\Theta)$ will tend to decrease with more data. $J_{validation}(\Theta)$ and $J_{training}(\Theta)$ will converge to a high value. **Getting more data will not help.**

### High Variance
If algorithm is suffering from variance (underfitting), it's easy to fit all the training set examples perfectly for a small set, so as the training set size increases $J_{training}(\Theta)$ increases, but will remain rather low. $J_{validation}(\Theta)$ will decrease as we add more examples, but **will remain high**.

The indicative diagnostic of a high variance problem is that we have a big gap between $J_{validation}(\Theta)$ and $J_{training}(\Theta)$ -- it seems likely that **if we add more data they will continue to converge.**

# Prioritizing What To Work On
* Collect lots of data
* Develop sophisticated features

## Recommended Approach
* Start with a simple algorithm. Implement and test it **on cross-validation data**.
  * If we develop new features by examining the test set, then we may end up choosing features that work well specifically for the test set, so $J_{test}(\Theta)$ is no longer a good estimate of how well we generalize to new examples.
* Plot learning curves to decide if more features or more data or something else is likely to help.
  * ^ this is avoiding premature optimization
* **Error analysis** -- look at the examples being misclassified -- perhaps this will inspire some new features
  * Classify the errors and/or try to identify what led to the errors
  * Determine numerical evaluation (e.g. cross-validation error) of algorithm's A/B performance
  
## Handling Skewed Classes
Taking the example of cancer prediction, perhaps we have an actual rate of cancer of 1% -- in that case we can write a stupid function like

```
def predict(x):
    return 0 # i.e. ignore x
```

which will have a 99% accuracy even though it isn't even a learning algorithm. In this case validation error rate is not a good evaluation measure. If we also use both **precision** and **recall** as our metric, and our algorithm scores high on both, it's much less likely that the algorithm is "cheating." The above stupid algorithm won't work because it will yield a recall = 0.

### Possible Outcomes For A Classifier
||Actual Class 1|Actual Class 0|
|:-:|:-:|:-:|
|**Predicted Class 1**|True Positive|False Positive|
|**Predicted Class 0**|False Negative|True Negative|

Note: usually $y = 1$ in the presence of the more rare class.

### Precision 
"Of all the patients for whom we predicted cancer, how many actually have it?"

$$
    \frac{\text{# of true positives}}{\text{# of predicted positives}} = \frac{TP}{TP + FP}
$$

### Recall
"Of all the patients who actually have cancer, how many did we predict?"
$$
    \frac{\text{# of true positives}}{\text{# of actual positives}} = \frac{TP}{TP + FN}
$$

### Trading Off Between Precision And Recall
In a classifier we use $h_{\theta}(x) \geq \text{threshold}$ to determine whether to predict 1 or 0.
* If we set a high threshold, we will maximize precision because we will have to be very sure of the patient having cancer to predict cancer, but we'll minimize recall because we'll predict negatives for some of the actual positives that we're not so sure about.
* If we set a low threshold, we will minimize precision because we'll predict positive for all sorts of examples, even those we're not really sure about, while maximizing recall because we'll be "casting a wide net."

#### How To Automatically Set The Threshold For A Classifier

Measure precision (P) and recall (R) on the cross validation set and choose the value of threshold which maximizes the [F Score](https://en.wikipedia.org/wiki/F1_score):

$$
    F_1 = 2\frac{PR}{P+R}
$$ 

This is particularly useful as a measure for skewed datasets.

# Large Data Rationale

Acquiring more data will not necessarily help if the features $x$ do not contain enough information to predict $y$ accurately (i.e. asking a human expert to make a prediction given these features will not yield a good result).

More parameters ensures low bias. A larger training set ensures low variance. Having both will often yield a high-performing learning algorithm.